<a href="https://colab.research.google.com/github/onlyabhilash/Spark_NLP/blob/main/spark-nlp_basics/spark_05_Text_classification_examples_in_SparkML_SparkNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification with Spark NLP

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.5.5

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
     |████████████████████████████████| 215.7 MB 59 kB/s 
     |████████████████████████████████| 197 kB 20.5 MB/s 
     |████████████████████████████████| 124 kB 9.0 MB/s 



<b>  if you want to work with Spark 2.3 </b>
```
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz

!tar xf spark-2.3.0-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-2.3.0-bin-hadoop2.7"
! java -version

import findspark
findspark.init()
from pyspark.sql import SparkSession

! pip install --ignore-installed -q spark-nlp==2.5.5

import sparknlp

spark = sparknlp.start(spark23=True)
```

In [ ]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import pandas as pd


In [ ]:

import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.5.5
Apache Spark version:  2.4.4


In [ ]:
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

--2022-03-18 05:56:48--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24032125 (23M) [text/plain]
Saving to: ‘news_category_train.csv’

news_category_train 100%[===================>]  22.92M  --.-KB/s    in 0.1s    

2022-03-18 05:56:50 (202 MB/s) - ‘news_category_train.csv’ saved [24032125/24032125]

--2022-03-18 05:56:50--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubuserco

In [ ]:
# newsDF = spark.read.parquet("data/news_category.parquet") >> if it is a parquet

newsDF = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")

newsDF.show(truncate=50)

+--------+--------------------------------------------------+
|category|                                       description|
+--------+--------------------------------------------------+
|Business| Short sellers, Wall Street's dwindling band of...|
|Business| Private investment firm Carlyle Group, which h...|
|Business| Soaring crude prices plus worries about the ec...|
|Business| Authorities have halted oil export flows from ...|
|Business| Tearaway world oil prices, toppling records an...|
|Business| Stocks ended slightly higher on Friday but sta...|
|Business| Assets of the nation's retail money market mut...|
|Business| Retail sales bounced back a bit in July, and n...|
|Business|" After earning a PH.D. in Sociology, Danny Baz...|
|Business| Short sellers, Wall Street's dwindling  band o...|
|Business| Soaring crude prices plus worries  about the e...|
|Business| OPEC can do nothing to douse scorching  oil pr...|
|Business| Non OPEC oil exporters should consider  increa...|
|Busines

In [ ]:
newsDF.show(truncate=50)

+--------+--------------------------------------------------+
|category|                                       description|
+--------+--------------------------------------------------+
|Business| Short sellers, Wall Street's dwindling band of...|
|Business| Private investment firm Carlyle Group, which h...|
|Business| Soaring crude prices plus worries about the ec...|
|Business| Authorities have halted oil export flows from ...|
|Business| Tearaway world oil prices, toppling records an...|
|Business| Stocks ended slightly higher on Friday but sta...|
|Business| Assets of the nation's retail money market mut...|
|Business| Retail sales bounced back a bit in July, and n...|
|Business|" After earning a PH.D. in Sociology, Danny Baz...|
|Business| Short sellers, Wall Street's dwindling  band o...|
|Business| Soaring crude prices plus worries  about the e...|
|Business| OPEC can do nothing to douse scorching  oil pr...|
|Business| Non OPEC oil exporters should consider  increa...|
|Busines

In [ ]:
newsDF.take(2)

[Row(category='Business', description=" Short sellers, Wall Street's dwindling band of ultra cynics, are seeing green again."),
 Row(category='Business', description=' Private investment firm Carlyle Group, which has a reputation for making well timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.')]

In [ ]:
from pyspark.sql.functions import col

newsDF.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|   World|30000|
|Sci/Tech|30000|
|Business|30000|
|  Sports|30000|
+--------+-----+



## Building Classification Pipeline

### LogReg with CountVectorizer

Tokenizer: Tokenization 

stopwordsRemover: Remove Stop Words

countVectors: Count vectors (“document-term vectors”)

In [ ]:
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer


In [ ]:
%%time

document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

countVectors = CountVectorizer(inputCol="token_features", outputCol="features", vocabSize=10000, minDF=5)

label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")

nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
           countVectors,
           label_stringIdx])

nlp_model = nlp_pipeline.fit(newsDF)

processed = nlp_model.transform(newsDF)

processed.count()

CPU times: user 691 ms, sys: 103 ms, total: 794 ms
Wall time: 1min 34s


In [ ]:
processed.select('description','token_features').show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                       description|                                    token_features|
+--------------------------------------------------+--------------------------------------------------+
| Short sellers, Wall Street's dwindling band of...|[short, seller, wall, street, dwindl, band, ult...|
| Private investment firm Carlyle Group, which h...|[privat, invest, firm, carlyl, group, reput, ma...|
| Soaring crude prices plus worries about the ec...|[soar, crude, price, plu, worri, economi, outlo...|
| Authorities have halted oil export flows from ...|[author, halt, oil, export, flow, main, pipelin...|
| Tearaway world oil prices, toppling records an...|[tearawai, world, oil, price, toppl, record, st...|
| Stocks ended slightly higher on Friday but sta...|[stock, end, slightli, higher, fridai, staye, n...|
| Assets of the nation's retail money market mut...|[asset, nati

In [ ]:
processed.select('token_features').take(2)

[Row(token_features=['short', 'seller', 'wall', 'street', 'dwindl', 'band', 'ultra', 'cynic', 'see', 'green']),
 Row(token_features=['privat', 'invest', 'firm', 'carlyl', 'group', 'reput', 'make', 'well', 'time', 'occasion', 'controversi', 'plai', 'defens', 'industri', 'quietli', 'place', 'bet', 'anoth', 'part', 'market'])]

In [ ]:
processed.select('features').take(2)

[Row(features=SparseVector(10000, {241: 1.0, 384: 1.0, 467: 1.0, 743: 1.0, 837: 1.0, 2233: 1.0, 3690: 1.0, 6224: 1.0, 6295: 1.0})),
 Row(features=SparseVector(10000, {26: 1.0, 38: 1.0, 46: 1.0, 68: 1.0, 117: 1.0, 155: 1.0, 182: 1.0, 197: 1.0, 245: 1.0, 304: 1.0, 320: 1.0, 407: 1.0, 427: 1.0, 621: 1.0, 867: 1.0, 2362: 1.0, 2834: 1.0, 2861: 1.0, 6877: 1.0}))]

In [ ]:
processed.select('description','features','label').show()

+--------------------+--------------------+-----+
|         description|            features|label|
+--------------------+--------------------+-----+
| Short sellers, W...|(10000,[241,384,4...|  1.0|
| Private investme...|(10000,[26,38,46,...|  1.0|
| Soaring crude pr...|(10000,[15,28,46,...|  1.0|
| Authorities have...|(10000,[0,32,35,4...|  1.0|
| Tearaway world o...|(10000,[1,2,11,28...|  1.0|
| Stocks ended sli...|(10000,[3,13,14,2...|  1.0|
| Assets of the na...|(10000,[0,4,10,15...|  1.0|
| Retail sales bou...|(10000,[0,1,10,15...|  1.0|
|" After earning a...|(10000,[98,99,125...|  1.0|
| Short sellers, W...|(10000,[241,384,4...|  1.0|
| Soaring crude pr...|(10000,[15,28,46,...|  1.0|
| OPEC can do noth...|(10000,[0,24,28,2...|  1.0|
| Non OPEC oil exp...|(10000,[0,21,28,3...|  1.0|
| WASHINGTON/NEW Y...|(10000,[2,4,13,14...|  1.0|
| The dollar tumbl...|(10000,[2,14,72,1...|  1.0|
|If you think you ...|(10000,[74,77,143...|  1.0|
|The purchasing po...|(10000,[46,54,167...|  1.0|


In [ ]:
# set seed for reproducibility
(trainingData, testData) = processed.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 84038
Test Dataset Count: 35962


In [ ]:
trainingData.printSchema()

root
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |   

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("description","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


+------------------------------+--------+------------------------------+-----+----------+
|                   description|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|Novell brings updated kerne...|Business|[0.9999995974765189,2.43882...|  1.0|       0.0|
| Cray taps Linux for more a...|Sci/Tech|[0.9999892931501291,5.27087...|  0.0|       0.0|
|F5 bolsters firewall family...|Sci/Tech|[0.9999765458165976,2.06665...|  0.0|       0.0|
| You Software Inc. announce...|Sci/Tech|[0.9999614023465399,1.09613...|  0.0|       0.0|
|Awarding the iMac G5 five s...|Sci/Tech|[0.9997506483003618,1.63025...|  0.0|       0.0|
|\\I've blogged before  abou...|Sci/Tech|[0.999593913378917,3.198068...|  0.0|       0.0|
| At a special music event o...|Sci/Tech|[0.9995669801377939,3.20953...|  0.0|       0.0|
| Not long ago most corporat...|Sci/Tech|[0.9995532840436969,2.59983...|  0.0|       0.0|
|IBM Corp.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

#evaluator.evaluate(predictions)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
y_true = predictions.select("label")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()


In [ ]:
y_pred.prediction.value_counts()

3.0    9385
0.0    9047
1.0    8865
2.0    8665
Name: prediction, dtype: int64

In [ ]:
cnf_matrix = confusion_matrix(list(y_true.label.astype(int)), list(y_pred.prediction.astype(int)))
cnf_matrix

array([[7791,  730,  311,  118],
       [ 882, 7657,  284,   85],
       [ 311,  426, 7992,  289],
       [  63,   52,   78, 8893]])

In [ ]:
print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.86      0.87      0.87      8950
         1.0       0.86      0.86      0.86      8908
         2.0       0.92      0.89      0.90      9018
         3.0       0.95      0.98      0.96      9086

    accuracy                           0.90     35962
   macro avg       0.90      0.90      0.90     35962
weighted avg       0.90      0.90      0.90     35962

0.8990879261442634


### LogReg with TFIDF

In [ ]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="token_features", outputCol="rawFeatures", numFeatures=10000)

idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

nlp_pipeline_tf = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
            hashingTF,
           idf,
           label_stringIdx])

nlp_model_tf = nlp_pipeline_tf.fit(newsDF)

processed_tf = nlp_model_tf.transform(newsDF)

processed_tf.count()


120000

In [ ]:
# set seed for reproducibility
processed_tf.select('description','features','label').show()

+--------------------+--------------------+-----+
|         description|            features|label|
+--------------------+--------------------+-----+
| Short sellers, W...|(10000,[25,625,66...|  1.0|
| Private investme...|(10000,[82,111,15...|  1.0|
| Soaring crude pr...|(10000,[410,1097,...|  1.0|
| Authorities have...|(10000,[1611,1637...|  1.0|
| Tearaway world o...|(10000,[1150,1427...|  1.0|
| Stocks ended sli...|(10000,[332,410,6...|  1.0|
| Assets of the na...|(10000,[1442,1788...|  1.0|
| Retail sales bou...|(10000,[25,117,97...|  1.0|
|" After earning a...|(10000,[114,643,7...|  1.0|
| Short sellers, W...|(10000,[25,625,66...|  1.0|
| Soaring crude pr...|(10000,[410,1097,...|  1.0|
| OPEC can do noth...|(10000,[616,904,1...|  1.0|
| Non OPEC oil exp...|(10000,[616,2224,...|  1.0|
| WASHINGTON/NEW Y...|(10000,[351,360,3...|  1.0|
| The dollar tumbl...|(10000,[359,456,9...|  1.0|
|If you think you ...|(10000,[1041,1564...|  1.0|
|The purchasing po...|(10000,[2198,4091...|  1.0|


In [ ]:
(trainingData, testData) = processed_tf.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 84038
Test Dataset Count: 35962


In [ ]:
lrModel_tf = lr.fit(trainingData)

predictions_tf = lrModel_tf.transform(testData)

predictions_tf.select("description","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


+------------------------------+--------+------------------------------+-----+----------+
|                   description|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|Novell brings updated kerne...|Business|[0.9999995229578786,2.86181...|  1.0|       0.0|
|F5 bolsters firewall family...|Sci/Tech|[0.9999631964683853,2.87934...|  0.0|       0.0|
|\\Sam blogs about his wiki ...|Sci/Tech|[0.9999591803496525,3.00816...|  0.0|       0.0|
| Cray taps Linux for more a...|Sci/Tech|[0.99988175049369,1.0958016...|  0.0|       0.0|
| At a special music event o...|Sci/Tech|[0.9997199938921201,2.03591...|  0.0|       0.0|
| You Software Inc. announce...|Sci/Tech|[0.9996235826906964,5.33405...|  0.0|       0.0|
|MOFFETT FIELD, CALIFORNIA -...|Sci/Tech|[0.9995490457561748,2.09063...|  0.0|       0.0|
|Sun Microsystems will integ...|Sci/Tech|[0.9990914778205654,2.87415...|  0.0|       0.0|
| Microsof

In [ ]:
y_true = predictions_tf.select("label")
y_true = y_true.toPandas()

y_pred = predictions_tf.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.85      0.85      0.85      8950
         1.0       0.85      0.85      0.85      8908
         2.0       0.91      0.88      0.90      9018
         3.0       0.94      0.96      0.95      9086

    accuracy                           0.89     35962
   macro avg       0.89      0.89      0.89     35962
weighted avg       0.89      0.89      0.89     35962

0.8877148100773038


### Random Forest with TFIDF

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions_rf = rfModel.transform(testData)


In [ ]:

predictions_rf.select("description","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                   description|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
| Microsoft Corp., hoping to...|Sci/Tech|[0.38025427051503,0.2615110...|  0.0|       0.0|
| You Software Inc. announce...|Sci/Tech|[0.37690896168759835,0.2343...|  0.0|       0.0|
|In a bid to dethrone Apple ...|Sci/Tech|[0.3744373024799478,0.23155...|  0.0|       0.0|
|Microsoft is expected to en...|Sci/Tech|[0.3742426420072038,0.23816...|  0.0|       0.0|
| On the face of things, Mic...|Sci/Tech|[0.37181825361445076,0.2616...|  0.0|       0.0|
| Microsoft Corp. launched a...|Sci/Tech|[0.3709883982206899,0.26160...|  0.0|       0.0|
|APPLE Computer has extended...|Sci/Tech|[0.3626937307330564,0.23037...|  0.0|       0.0|
|Microsoft will begin offeri...|Sci/Tech|[0.362291694911747,0.221690...|  0.0|       0.0|
|com Decem

In [ ]:
y_true = predictions_rf.select("label")
y_true = y_true.toPandas()

y_pred = predictions_rf.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.73      0.63      0.68      8950
         1.0       0.74      0.67      0.71      8908
         2.0       0.81      0.74      0.77      9018
         3.0       0.69      0.91      0.79      9086

    accuracy                           0.74     35962
   macro avg       0.75      0.74      0.74     35962
weighted avg       0.75      0.74      0.74     35962

0.739669651298593


## LogReg with Spark NLP Glove Word Embeddings

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

glove_embeddings = WordEmbeddingsModel().pretrained() \
 .setInputCols(["document",'cleanTokens'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_sentence_embeddings"]) \
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)

explodeVectors = SQLTransformer(statement=
      "SELECT EXPLODE(finished_sentence_embeddings) AS features, * FROM __THIS__")

label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")


nlp_pipeline_w2v = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            glove_embeddings,
            embeddingsSentence,
            embeddings_finisher,
            explodeVectors,
           label_stringIdx])

nlp_model_w2v = nlp_pipeline_w2v.fit(newsDF)

processed_w2v = nlp_model_w2v.transform(newsDF)

processed_w2v.count()


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


120000

In [ ]:
#processed_w2v.select('finished_embeddings').take(1)

In [ ]:
#processed_w2v.select("finished_embeddings").show(1)

In [ ]:
processed_w2v.select('finished_sentence_embeddings').take(1)

[Row(finished_sentence_embeddings=[DenseVector([-0.1557, 0.196, 0.1099, -0.3089, 0.16, 0.1672, -0.4649, -0.1101, -0.053, -0.1551, 0.0327, 0.0772, 0.1494, -0.1865, 0.1155, -0.0597, 0.0234, -0.0451, 0.2361, -0.0089, 0.3358, 0.0444, 0.0088, -0.1453, 0.2289, 0.0914, -0.1665, -0.3726, 0.1892, 0.121, 0.1993, -0.0239, -0.1346, 0.1159, 0.2086, 0.1285, 0.068, 0.1372, 0.3153, -0.1934, 0.0257, -0.226, -0.0984, 0.1139, 0.1413, -0.3743, 0.072, 0.1403, 0.251, -0.3106, 0.1709, -0.0697, -0.0554, 0.5123, -0.1873, -1.7784, 0.0295, 0.1014, 0.9268, 0.2129, -0.1354, 0.5739, -0.0679, 0.461, 0.4216, 0.0225, 0.4456, -0.2462, 0.1411, -0.3258, 0.0025, 0.0114, -0.3895, -0.1106, -0.261, 0.0147, 0.0781, 0.1268, -0.2042, -0.2278, 0.5096, 0.1539, -0.3515, -0.0102, -0.7003, -0.3872, -0.1668, -0.2405, -0.0766, 0.1396, -0.0592, -0.1568, -0.1606, -0.1371, -0.684, -0.2549, -0.1541, 0.1536, 0.2715, 0.3342])])]

In [ ]:
# IF SQLTransformer IS NOT USED INSIDE THE PIPELINE, WE CAN EXPLODE OUTSIDE
from pyspark.sql.functions import explode

# processed_w2v= processed_w2v.withColumn("features", explode(processed_w2v.finished_sentence_embeddings))

In [ ]:
processed_w2v.select("features").take(1)

[Row(features=DenseVector([-0.1557, 0.196, 0.1099, -0.3089, 0.16, 0.1672, -0.4649, -0.1101, -0.053, -0.1551, 0.0327, 0.0772, 0.1494, -0.1865, 0.1155, -0.0597, 0.0234, -0.0451, 0.2361, -0.0089, 0.3358, 0.0444, 0.0088, -0.1453, 0.2289, 0.0914, -0.1665, -0.3726, 0.1892, 0.121, 0.1993, -0.0239, -0.1346, 0.1159, 0.2086, 0.1285, 0.068, 0.1372, 0.3153, -0.1934, 0.0257, -0.226, -0.0984, 0.1139, 0.1413, -0.3743, 0.072, 0.1403, 0.251, -0.3106, 0.1709, -0.0697, -0.0554, 0.5123, -0.1873, -1.7784, 0.0295, 0.1014, 0.9268, 0.2129, -0.1354, 0.5739, -0.0679, 0.461, 0.4216, 0.0225, 0.4456, -0.2462, 0.1411, -0.3258, 0.0025, 0.0114, -0.3895, -0.1106, -0.261, 0.0147, 0.0781, 0.1268, -0.2042, -0.2278, 0.5096, 0.1539, -0.3515, -0.0102, -0.7003, -0.3872, -0.1668, -0.2405, -0.0766, 0.1396, -0.0592, -0.1568, -0.1606, -0.1371, -0.684, -0.2549, -0.1541, 0.1536, 0.2715, 0.3342]))]

In [ ]:
processed_w2v.select("features").take(1)

[Row(features=DenseVector([-0.1557, 0.196, 0.1099, -0.3089, 0.16, 0.1672, -0.4649, -0.1101, -0.053, -0.1551, 0.0327, 0.0772, 0.1494, -0.1865, 0.1155, -0.0597, 0.0234, -0.0451, 0.2361, -0.0089, 0.3358, 0.0444, 0.0088, -0.1453, 0.2289, 0.0914, -0.1665, -0.3726, 0.1892, 0.121, 0.1993, -0.0239, -0.1346, 0.1159, 0.2086, 0.1285, 0.068, 0.1372, 0.3153, -0.1934, 0.0257, -0.226, -0.0984, 0.1139, 0.1413, -0.3743, 0.072, 0.1403, 0.251, -0.3106, 0.1709, -0.0697, -0.0554, 0.5123, -0.1873, -1.7784, 0.0295, 0.1014, 0.9268, 0.2129, -0.1354, 0.5739, -0.0679, 0.461, 0.4216, 0.0225, 0.4456, -0.2462, 0.1411, -0.3258, 0.0025, 0.0114, -0.3895, -0.1106, -0.261, 0.0147, 0.0781, 0.1268, -0.2042, -0.2278, 0.5096, 0.1539, -0.3515, -0.0102, -0.7003, -0.3872, -0.1668, -0.2405, -0.0766, 0.1396, -0.0592, -0.1568, -0.1606, -0.1371, -0.684, -0.2549, -0.1541, 0.1536, 0.2715, 0.3342]))]

In [ ]:
processed_w2v.select('description','features','label').show()


+--------------------+--------------------+-----+
|         description|            features|label|
+--------------------+--------------------+-----+
| Short sellers, W...|[-0.1556767076253...|  1.0|
| Private investme...|[-0.0144653050228...|  1.0|
| Soaring crude pr...|[0.10348732769489...|  1.0|
| Authorities have...|[-0.0355810523033...|  1.0|
| Tearaway world o...|[0.00647281948477...|  1.0|
| Stocks ended sli...|[0.20069395005702...|  1.0|
| Assets of the na...|[0.38012433052062...|  1.0|
| Retail sales bou...|[0.20352847874164...|  1.0|
|" After earning a...|[0.13536226749420...|  1.0|
| Short sellers, W...|[-0.1556767076253...|  1.0|
| Soaring crude pr...|[0.10348732769489...|  1.0|
| OPEC can do noth...|[0.20307321846485...|  1.0|
| Non OPEC oil exp...|[0.09010648727416...|  1.0|
| WASHINGTON/NEW Y...|[0.10887209326028...|  1.0|
| The dollar tumbl...|[0.05723679438233...|  1.0|
|If you think you ...|[0.11463439464569...|  1.0|
|The purchasing po...|[0.05890964344143...|  1.0|


In [ ]:
# set seed for reproducibility
(trainingData, testData) = processed_w2v.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 84038
Test Dataset Count: 35962


In [ ]:
from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

testData = testData.where(num_nonzeros("features") != 0)

In [ ]:
testData.count()

35962

In [ ]:
lrModel_w2v = lr.fit(trainingData)

In [ ]:
predictions_w2v = lrModel_w2v.transform(testData)

predictions_w2v.select("description","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


+------------------------------+--------+------------------------------+-----+----------+
|                   description|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|The KDE Project has release...|Sci/Tech|[0.9993026934526253,5.26811...|  0.0|       0.0|
| Users can now access searc...|Sci/Tech|[0.9984190388827833,0.00112...|  0.0|       0.0|
|" The Xbox version of ""Doom 3|Sci/Tech|[0.9978700979929389,8.61016...|  0.0|       0.0|
|" The Xbox version of ""Doom 3|Sci/Tech|[0.9978700979929389,8.61016...|  0.0|       0.0|
|" The Xbox version of ""Doom 3|Sci/Tech|[0.9978700979929389,8.61016...|  0.0|       0.0|
|With Google Desktop Search,...|Sci/Tech|[0.9966360633091306,0.00214...|  0.0|       0.0|
|Google has finally announce...|Business|[0.996501742592862,0.002382...|  1.0|       0.0|
|But users of the popular co...|Sci/Tech|[0.9963135491275907,0.00278...|  0.0|       0.0|
|" Users o

In [ ]:
y_true = predictions_w2v.select("label")
y_true = y_true.toPandas()

y_pred = predictions_w2v.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.82      0.81      0.81      8902
         1.0       0.82      0.83      0.82      9072
         2.0       0.88      0.87      0.87      9006
         3.0       0.93      0.95      0.94      8982

    accuracy                           0.86     35962
   macro avg       0.86      0.86      0.86     35962
weighted avg       0.86      0.86      0.86     35962

0.8633835715477448


In [ ]:
processed_w2v.select('description','cleanTokens.result').show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                       description|                                            result|
+--------------------------------------------------+--------------------------------------------------+
| Short sellers, Wall Street's dwindling band of...|[Short, sellers, Wall, Streets, dwindling, band...|
| Private investment firm Carlyle Group, which h...|[Private, investment, firm, Carlyle, Group, rep...|
| Soaring crude prices plus worries about the ec...|[Soaring, crude, prices, plus, worries, economy...|
| Authorities have halted oil export flows from ...|[Authorities, halted, oil, export, flows, main,...|
| Tearaway world oil prices, toppling records an...|[Tearaway, world, oil, prices, toppling, record...|
| Stocks ended slightly higher on Friday but sta...|[Stocks, ended, slightly, higher, Friday, staye...|
| Assets of the nation's retail money market mut...|[Assets, nat

## LogReg with Spark NLP Bert Embeddings

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

bert_embeddings = BertEmbeddings\
 .pretrained('bert_base_cased', 'en') \
 .setInputCols(["document",'cleanTokens'])\
 .setOutputCol("bert")\
 .setCaseSensitive(False)\
 .setPoolingLayer(0)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_sentence_embeddings"]) \
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)

label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")


nlp_pipeline_bert = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            bert_embeddings,
            embeddingsSentence,
            embeddings_finisher,
           label_stringIdx])

nlp_model_bert = nlp_pipeline_bert.fit(newsDF)

processed_bert = nlp_model_bert.transform(newsDF)

processed_bert.count()


bert_base_cased download started this may take some time.
Approximate size to download 389.2 MB
[OK!]


120000

In [ ]:
from pyspark.sql.functions import explode

processed_bert= processed_bert.withColumn("features", explode(processed_bert.finished_sentence_embeddings))

processed_bert.select('description','features','label').show()


+--------------------+--------------------+-----+
|         description|            features|label|
+--------------------+--------------------+-----+
| Short sellers, W...|[-0.2046198099851...|  1.0|
| Private investme...|[0.05162287876009...|  1.0|
| Soaring crude pr...|[-0.3564318418502...|  1.0|
| Authorities have...|[0.07367337495088...|  1.0|
| Tearaway world o...|[0.27237433195114...|  1.0|
| Stocks ended sli...|[-0.0425911881029...|  1.0|
| Assets of the na...|[0.12972426414489...|  1.0|
| Retail sales bou...|[0.19162152707576...|  1.0|
|" After earning a...|[0.07279182225465...|  1.0|
| Short sellers, W...|[-0.2046198099851...|  1.0|
| Soaring crude pr...|[-0.3564318418502...|  1.0|
| OPEC can do noth...|[-0.0903253182768...|  1.0|
| Non OPEC oil exp...|[0.33865353465080...|  1.0|
| WASHINGTON/NEW Y...|[0.26189467310905...|  1.0|
| The dollar tumbl...|[0.32719594240188...|  1.0|
|If you think you ...|[0.17194975912570...|  1.0|
|The purchasing po...|[-0.1187076270580...|  1.0|


In [ ]:
# set seed for reproducibility
(trainingData, testData) = processed_bert.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 84038
Test Dataset Count: 35962


In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(trainingData)


KeyboardInterrupt: ignored

In [ ]:
from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

testData = testData.where(num_nonzeros("features") != 0)

In [ ]:
predictions = lrModel.transform(testData)

predictions.select("description","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd

df = predictions.select('description','category','label','prediction').toPandas()

print(classification_report(df.label, df.prediction))
print(accuracy_score(df.label, df.prediction))

## LogReg with ELMO Embeddings

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

elmo_embeddings = ElmoEmbeddings.load('/Users/vkocaman/cache_pretrained/elmo_en_2.4.0_2.4_1580488815299')\
      .setPoolingLayer("word_emb")\
      .setInputCols(["document",'cleanTokens'])\
      .setOutputCol("elmo")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "elmo"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_sentence_embeddings"]) \
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)

label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")


nlp_pipeline_elmo = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            elmo_embeddings,
            embeddingsSentence,
            embeddings_finisher,
           label_stringIdx])

nlp_model_elmo = nlp_pipeline_elmo.fit(newsDF)

processed_elmo = nlp_model_elmo.transform(newsDF)

processed_elmo.count()


In [ ]:
(trainingData, testData) = newsDF.randomSplit([0.7, 0.3], seed = 100)

In [ ]:
processed_trainingData = nlp_model_elmo.transform(trainingData)

processed_trainingData.count()

In [ ]:
processed_testData = nlp_model_elmo.transform(testData)

processed_testData.count()

In [ ]:
processed_trainingData.columns

In [ ]:


processed_testData= processed_testData.withColumn("features", explode(processed_testData.finished_sentence_embeddings))

processed_trainingData= processed_trainingData.withColumn("features", explode(processed_trainingData.finished_sentence_embeddings))


In [ ]:
from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

processed_testData = processed_testData.where(num_nonzeros("features") != 0)

In [ ]:
%%time

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(processed_trainingData)


In [ ]:
processed_trainingData.columns

In [ ]:

predictions = lrModel.transform(processed_testData)

predictions.select("description","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


In [ ]:
df = predictions.select('description','category','label','prediction').toPandas()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(df.label, df.prediction))
print(accuracy_score(df.label, df.prediction))

## LogReg with Universal Sentence Encoder

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")

In [ ]:

useEmbeddings = UniversalSentenceEncoder.load('/Users/vkocaman/cache_pretrained/tfhub_use_en_2.4.0_2.4_1580582893733')\
      .setInputCols("document")\
      .setOutputCol("use_embeddings")

In [ ]:

embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["use_embeddings"]) \
    .setOutputCols(["finished_use_embeddings"]) \
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)

label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")

use_pipeline = Pipeline(
      stages=[
        document_assembler,
          useEmbeddings,
        embeddings_finisher,
      label_stringIdx]
      )

In [ ]:
use_df = use_pipeline.fit(newsDF).transform(newsDF)

In [ ]:
use_df.select('finished_use_embeddings').show(3)

In [ ]:
from pyspark.sql.functions import explode

use_df= use_df.withColumn("features", explode(use_df.finished_use_embeddings))

In [ ]:
use_df.show(2)

In [ ]:
# set seed for reproducibility
(trainingData, testData) = use_df.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("description","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


In [ ]:
df = predictions.select('description','category','label','prediction').toPandas()
#df['result'] = df['result'].apply(lambda x: x[0])


In [ ]:
df.head()

In [ ]:

print(classification_report(df.label, df.prediction))
print(accuracy_score(df.label, df.prediction))

### train on entire dataset

In [ ]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(use_df)

In [ ]:

test_df = spark.read.parquet("data/news_category_test.parquet")


In [ ]:
test_df = use_pipeline.fit(test_df).transform(test_df)

In [ ]:
test_df= test_df.withColumn("features", explode(test_df.finished_use_embeddings))

In [ ]:
test_df.show(2)

In [ ]:
predictions = lrModel.transform(test_df)

In [ ]:
df = predictions.select('description','category','label','prediction').toPandas()

In [ ]:
df['label'] = df.category.replace({'World':2.0,
                    'Sports':3.0,
                    'Business':0.0,
                    'Sci/Tech':1.0})

In [ ]:
df.head()

In [ ]:
print(classification_report(df.label, df.prediction))
print(accuracy_score(df.label, df.prediction))

## Spark NLP Licensed DocClassifier

In [ ]:
from sparknlp_jsl.annotator import *

In [ ]:
# set seed for reproducibility
(trainingData, testData) = newsDF.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

In [ ]:

document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

logreg = DocumentLogRegClassifierApproach()\
      .setInputCols(["stem"])\
      .setLabelCol("category")\
      .setOutputCol("prediction")

nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
           logreg])

nlp_model = nlp_pipeline.fit(trainingData)

processed = nlp_model.transform(testData)

processed.count()

In [ ]:
processed.select('description','category','prediction.result').show(truncate=50)

In [ ]:
processed.select('description','prediction.result').show(truncate=50)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd

In [ ]:
df = processed.select('description','category','prediction.result').toPandas()

In [ ]:
df.head()

In [ ]:
df.result[0][0]

In [ ]:
df = processed.select('description','category','prediction.result').toPandas()
df['result'] = df['result'].apply(lambda x: x[0])

In [ ]:
df.head()

In [ ]:

df = processed.select('description','category','prediction.result').toPandas()
df['result'] = df['result'].apply(lambda x: x[0])

print(classification_report(df.category, df.result))
print(accuracy_score(df.category, df.result))

# ClassifierDL

In [ ]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.load('/Users/vkocaman/cache_pretrained/tfhub_use_en_2.4.4_2.4_1583158595769')\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

In [ ]:
# set seed for reproducibility
(trainingData, testData) = newsDF.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

In [ ]:
pipelineModel = pipeline.fit(trainingData)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

df = pipelineModel.transform(testDataset).select('category','description',"class.result").toPandas()

df['result'] = df['result'].apply(lambda x: x[0])

print(classification_report(df.category, df.result))
print(accuracy_score(df.category, df.result))

## Loading the trained classifier from disk

In [ ]:
classsifierdlmodel = ClassifierDLModel.load('classifierDL_model_20200317_5e')
 

In [ ]:
import sparknlp
sparknlp.__path__

In [ ]:
 .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("data/news_category_train.csv")

In [ ]:
trainDataset.count()

In [ ]:
trainingData.count()

In [ ]:
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")


sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

use = UniversalSentenceEncoder.load('/Users/vkocaman/cache_pretrained/tfhub_use_en_2.4.4_2.4_1583158595769')\
 .setInputCols(["sentence"])\
 .setOutputCol("sentence_embeddings")

classsifierdlmodel = ClassifierDLModel.load('classifierDL_model_20200317_5e')

pipeline = Pipeline(
    stages = [
        document,
        sentence,
        use,
        classsifierdlmodel
    ])

In [ ]:
pipeline.fit(testData.limit(1)).transform(testData.limit(10)).select('category','description',"class.result").show(10, truncate=50)

In [ ]:
lm = LightPipeline(pipeline.fit(testDataset.limit(1)))
lm.annotate('In its first two years, the UK dedicated card companies have surge')

In [ ]:
text='''
Fearing the fate of Italy, the centre-right government has threatened to be merciless with those who flout tough restrictions. As of Wednesday it will also include all shops being closed across Greece, with the exception of supermarkets. Banks, pharmacies, pet-stores, mobile phone stores, opticians, bakers, mini-markets, couriers and food delivery outlets are among the few that will also be allowed to remain open.
'''

In [ ]:
lm = LightPipeline(pipeline.fit(testDataset.limit(1)))

lm.annotate(text)

# Classifier DL + Glove + Basic text processing

In [ ]:
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["token"]) \
    .setOutputCol("lemma")

lemma_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            lemma,
           glove_embeddings])

In [ ]:
lemma_pipeline.fit(trainingData.limit(1000)).transform(trainingData.limit(1000)).show(truncate=30)

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
 .setInputCols(["document",'lemma'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(10)\
  .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

In [ ]:
!rm -rf classifier_dl_pipeline_glove

In [ ]:
clf_pipelineModel.save('classifier_dl_pipeline_glove')

In [ ]:
clf_pipelineModel = clf_pipeline.fit(trainingData)

In [ ]:
df = clf_pipelineModel.transform(testDataset).select('category','description',"class.result").toPandas()

df['result'] = df['result'].apply(lambda x: x[0])

print(classification_report(df.category, df.result))

print(accuracy_score(df.category, df.result))

In [ ]:
!cd data && ls -l

In [ ]:
import pandas as pd
import

In [ ]:
news_df = newsDF.toPandas()

In [ ]:
news_df.head()

In [ ]:
news_df.to_csv('data/news_dataset.csv', index=False)

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
 .setInputCols(["document",'lemma'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)

txt_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence])

In [ ]:

txt_pipelineModel = txt_pipeline.fit(testData.limit(1))

In [ ]:
txt_pipelineModel.save('text_prep_pipeline_glove')

In [ ]:
df.head()